<a href="https://colab.research.google.com/github/lianghu19/MAT-280-image-compression-project/blob/main/MAT_280_HP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Deconstructing Images: A Visual Analysis of Low-Rank Approximation with SVD**

Institution: Durham Technical Community College    
Instructor: Chris Mansfield    
Student: Zhiliang Hu



## Project goals:

The primary objective of this project is to visualize the procedural steps of Singular Value Decomposition (SVD). The challenge arises from the fact that these methods are most powerfully applied in high-dimensional data compression, an environment that is impossible to visualize directly within two or three dimensions. Hence, a purely abstract mathematical visualization or a standard high-dimensional dataset visualization would fail to provide an intuitive understanding.To bridge this gap, this project will leverage image compression as a practical and visually tangible application. An image, as a 2D matrix of pixel values, serves as a perfect medium.This approach will allow us to deconstruct the SVD process in an intuitive context, illustrate the concept of low-rank approximation by reconstructing an image using a subset of singular values, and visually connect the core mathematical components of SVD (the matrices $U$, $S$, and $V^T$) to tangible features and patterns within the image itself.

## Background:

SVD/PCA has many applications in image compression, but due to their large file sizes, the mainstream methods are JPEG (A lossy compression method that reduces file size by lossing some details, commonly used for photographs) and PNG (A lossless compression format, suitable for graphics and text, usually with larger file sizes). SVD/PCA offer unique advantages in scientific image analysis, where efficiency and cost are less important than the quality of results.

'Vectors' in this project refers to numerical vectors/matrices, not vector graphics. We will be using Numpy for mathematical operation, Pillow for image loading and processing, and Matplotlib for visualization.



## Process:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


### Step 1: Loading the image and convert to grayscale

In [ ]:
from PIL import Image
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt


image_path='/content/drive/MyDrive/DT_photo.png'

try:
  img=Image.open(image_path)
  print("Image import successful!")
  display(img)
  # Use display() because .show() method in pillow doesn't have direct access to an external image viewer since Colab runs in a cloud environment

except FileNotFoundError:
  print(f"Error: Image not found at '{image_path}'.")

### Convert the picture to grayscale

In [ ]:
grayscale_image=img.convert('L')
print("Image convert successful!")
display(grayscale_image)
# grayscale_image.save(grayscale_dt.png)

### Why convert to grayscale?

This process is necessary for simplicity. A typical image is composed of pixels defined by a combinatioon of three primary color channels (RGB), and this required the image to not just has a single 2D grid of pixels but three 2D matrices stacked on top of each other (Basically 3D：
$R = U_R \Sigma_R V_R^T$ $G = U_G \Sigma_G V_G^T$ $B = U_B \Sigma_B V_B^T$). When a colored image is converted to grayscale, the color information is removed, and each pixel is then represented by a single intensity value, typically ranging from black (0) to white (255) (uint8) or 0.0 to 1.0 after normalization (which is what our project does).

SVD and PCA are typically applied to single 2D matrices. By converting to grayscale, the problem thereby simplified from dealing with multiple color channels (Three matrices, three SVD) to analyzing one single intensity, or brightness, channel.

### Step 2: Convert image (pixels) to matrices (arrays)

In [ ]:
img_array=np.array(grayscale_image)
print(f'Data type {img_array.dtype}')
print(f'The maximum {img_array.max()}')
print(f'The minimum {img_array.min()}')

plt.figure(figsize=(10, 7))
im = plt.imshow(img_array, cmap='gray', vmin=0, vmax=255)
plt.title("Grayscale Image with Axes", fontsize=16)
plt.xlabel("Column Index (Width)", fontsize=12)
plt.ylabel("Row Index (Height)", fontsize=12)
plt.colorbar(im, label="Pixel Brightness (0=Black, 255=White)")
plt.show()

This is not ideal because the uint8 format (an 8-bit unsigned integer) can only store non-negative integers within the 0-255 range.SVD, however, involves many calculations with decimal scaling factors; uint8 would forcibly truncate these decimals into integers, creating significant error. Also, when SVD decomposes the image, it must create orthogonal vectors in the U and V matrices (to capture important, independent, and non-redundant features), which inevitably produces negative numbers (to make dot product between eigenvectors equals to 0). The uint8 format cannot process negative numbers.Furthermore, during the calculations, if pixel brightnesses are relatively large (e.g., 200), their multiplication will be much greater than 255. uint8 cannot store this, leading to numerical overflow.

      
                                                                                               
Therefore, we need numpy to cast the values and normalized them into 0-1 scale for calculation efficiency:

In [ ]:
img_float=img_array/255.0 # implicit casting using the float type 255.0
print("After normlization:")
print(f"dtype: {img_float.dtype}")
print(f"shape: {img_float.shape}")
print(f"max/min: {img_float.min()} / {img_float.max()}")

# Here, img_float is our Matrix A

### Step 3: Performing SVD

Here we get the result of U, V, and Singular values

In [ ]:
try:
    U_full, s_full, Vh_full = np.linalg.svd(img_float, full_matrices=True)

    print("Full SVD calculated successfully.")
    print("Shape of returned matrices:")
    print(f"  U_full.shape:  {U_full.shape}   <--- (m, m)")
    print(f"  s_full.shape:  {s_full.shape}    <--- This is a 1D vector, length n")
    print(f"  Vh_full.shape: {Vh_full.shape}  <--- (n, n)")

except np.linalg.LinAlgError as e:
    print(f"Full SVD failed: {e}")


print("-------------------------------------------------------------")

try:
    U_reduced, s_reduced, Vh_reduced = np.linalg.svd(img_float, full_matrices=False)

    print("Reduced SVD calculated successfully.")
    print("Shape of returned matrices:")
    print(f"  U_reduced.shape:  {U_reduced.shape}   <--- (m, n)")
    print(f"  s_reduced.shape:  {s_reduced.shape}    <--- This is a 1D vector, length n")
    print(f"  Vh_reduced.shape: {Vh_reduced.shape}  <--- (n, n)")

except np.linalg.LinAlgError as e:
    print(f"Reduced SVD failed: {e}")

# The reduced matrix is more efficient than the former one

In [ ]:
num_patterns_to_show = 4

m, _ = U_reduced.shape  # m = 448
_, n = Vh_reduced.shape # n = 672

print(f"\n--- Visualizing U: The 'Eigen-row' Patterns (first {num_patterns_to_show}) ---")

fig, axes = plt.subplots(1, num_patterns_to_show, figsize=(12, 7),dpi=300) # (1 row, 4 columns)
fig.suptitle(f"U: Full-Size Eigen-row Patterns (Columns of U)\n(Shape: {m}x1 strips)", fontsize=16, y=1.05)

for k in range(num_patterns_to_show):
    u_k_vector = U_reduced[:, k] # [row_slicing, column_slicing]
    u_k_image = u_k_vector.reshape(m, 1) # .reshape(row, column)
    vmax = np.max(np.abs(u_k_image)) #  vmin=-vmax，to make the color scale symmetric
    ax = axes[k]
    ax.imshow(u_k_image, cmap='seismic', aspect=0.01, vmin=-vmax, vmax=vmax)
    # Red is positive and blue is negative
    # "Aspect" argument determines the proportional relationship between the width and height of the pixels or the overall image within the axes. aspect = (y-pixel-height) / (x-pixel-width), the smaller, the more squashed of the plot vertically.
    ax.set_title(f"$u_{k+1}$ (k={k})")
    ax.set_xticks([])
    ax.set_ylabel(f"Row Index (0-{m-1})")

plt.tight_layout()
plt.show()


print(f"\n--- Visualizing Vh: The 'Eigen-column' Patterns (first {num_patterns_to_show}) ---")


fig, axes = plt.subplots(num_patterns_to_show, 1, figsize=(15, 8),dpi=300)
fig.suptitle(f"Vh: Full-Size Eigen-column Patterns (Rows of Vh)\n(Shape: 1x{n} strips)", fontsize=16, y=1.02)
for k in range(num_patterns_to_show):
    v_k_T_vector = Vh_reduced[k, :]
    v_k_T_image = v_k_T_vector.reshape(1, n)
    vmax = np.max(np.abs(v_k_T_image))
    ax = axes[k]
    ax.imshow(v_k_T_image, cmap='seismic', aspect=20, vmin=-vmax, vmax=vmax)
    ax.set_title(f"$v_{k+1}^T$ (k={k})")
    ax.set_yticks([])
    ax.set_xlabel(f"Column Index (0-{n-1})")


plt.tight_layout(pad=1.5)
plt.show()

Explantation needed

### Visualizing Singular Value Significance (Scree Plot)

This plot shows the relative importance captured by each singular value. The sharp drop off, as we will see, indicates that the first few singular values are dominantly important. This tells why we can effectively compress the image by keeping only the first k component, as the remaining values contribute very little to the overall structure.

There will be two plots:

Linear Scale: To show the dramatic drop-off of the first few values.

Logarithmic Scale: To better visualize the "tail" and see how the smaller values decay (to help distinguish small differences between different number of K).

In [ ]:
import matplotlib.pyplot as plt
print("\nVisualizing Singular Values---")
# Plot 1: Standard Linear Scale
plt.figure(figsize=(8, 5), dpi=200)
plt.style.use('seaborn-v0_8-whitegrid')
plt.plot(s_reduced, marker='o', markersize=2, linestyle='-')
plt.title("Scree Plot: Singular Values (Linear Scale)", fontsize=18)
plt.xlabel("Singular Value Rank ($k$)", fontsize=14)
plt.ylabel("Magnitude ($σ_k$)", fontsize=14)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.xlim(-10, len(s_reduced) + 10)
plt.show()


# Plot 2: Logarithmic Scale
plt.figure(figsize=(8, 5),dpi=200)
plt.style.use('seaborn-v0_8-whitegrid')
plt.plot(s_reduced, marker='o', markersize=2, linestyle='-')

# Set y-axis to log scale
plt.yscale('log')
plt.title("Scree Plot: Singular Values (Log Scale)", fontsize=18)
plt.xlabel("Singular Value Rank ($k$)", fontsize=14)
plt.ylabel("Magnitude ($σ_k$, log scale)", fontsize=14)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.xlim(-10, len(s_reduced) + 10) # Show all singular values
plt.show()

### Calculate similarity maps, using matplotlib and plotly, respectively

In [ ]:
# This matrix shows how similar each row of pixels is to every other row
MMT = img_float @ img_float.T # S_l
# col_i[0]*col_j[0] + col_i[1]*col_j[1] + ... + col_i[447]*col_j[447]

# Visualize this new 448x448 matrix
plt.figure(figsize=(5, 5), dpi=200)
plt.style.use('default')
im = plt.imshow(MMT, cmap='hot')
plt.title("Row-Row Similarity Map", fontsize=16)
plt.xlabel("Row Index (j)", fontsize=12)
plt.ylabel("Row Index (i)", fontsize=12)
plt.colorbar(im, label="Similarity Score") # Add a color bar to show the scale
plt.show()

MTM = img_float.T @ img_float # S_r
plt.figure(figsize=(5, 5), dpi=200)
plt.style.use('default')
im = plt.imshow(MTM, cmap='hot')
plt.title("Column-Column Similarity Map", fontsize=16)
plt.xlabel("Column Index (j)", fontsize=12)
plt.ylabel("Column Index (i)", fontsize=12)
plt.colorbar(im, label="Similarity Score") # Add a color bar to show the scale
plt.show()


In [ ]:
import plotly.express as px

fig_row = px.imshow(
    MMT,
    title="<b> Row Similarity (M @ M.T)</b>",
    color_continuous_scale='hot',
)
fig_row.update_layout(xaxis_title="Row Index", yaxis_title="Row Index")
fig_row.show()


fig_column = px.imshow(
    MTM,
    title="<b> Column Similarity (M.T @ M)</b>",
    color_continuous_scale='hot',
)
fig_column.update_layout(xaxis_title="Coluumn Index", yaxis_title="Coluumn Index")
fig_column.show()

Row Similarity Map (M @ M.T): This (448, 448) heatmap shows the dot product between every pair of horizontal rows.

The large, bright square in the top-left corner (approx. 0-150) shows that the top rows of the original image are all highly similar. This corresponds to the bright sky, where rows have both a similar pattern (all bright) and a high magnitude (high brightness).

The dark region in the bottom-right corner corresponds to the shadows. Even if these shadow-rows have a similar pattern, their magnitude (brightness) is very low, resulting in a low dot product.

Column Similarity Map (M.T @ M): This (672, 672) heatmap shows the dot product between every pair of vertical columns.

The extremely bright square in the top-right corner (approx. 550-672) corresponds perfectly to the far-right columns of the original image, which are almost entirely bright sky (high magnitude, similar direction).

The bright, cross-like pattern in the center (approx. 300-500) corresponds to the large tree. These columns have high-energy (bright) pixels from the sun and sky, giving them a high dot product with each other and with the other bright-sky columns.

These similarity maps are based on the dot product, not pure pattern matching. Therefore, a bright spot on the heatmap signifies that two rows (or columns) are highly similar because they capture the combined impact of both vector direction (the pattern of pixels) and vector magnitude (the overall brightness or "energy" of that line).

### Calculating Cosine similarity

Cosine similarity, on the other hand, only measures the directonal simiarity, eliminating the effects of magnitude (brightness in this case).

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

row_cos_sim = cosine_similarity(img_float)
col_cos_sim = cosine_similarity(img_float.T)
print(f"Row Similarity Matrix Shape: {row_cos_sim.shape}")
print(f"Column Similarity Matrix Shape: {col_cos_sim.shape}")
plt.figure(figsize=(16, 7), dpi=200)
plt.style.use('default')


ax1 = plt.subplot(1, 2, 1)
im1 = ax1.imshow(row_cos_sim, cmap='hot', vmin=0, vmax=1)
ax1.set_title("Row-Row Cosine Similarity", fontsize=14)
ax1.set_xlabel("Row Index")
ax1.set_ylabel("Row Index")
plt.colorbar(im1, ax=ax1, label="Similarity Score (0 to 1)")

ax2 = plt.subplot(1, 2, 2)
im2 = ax2.imshow(col_cos_sim, cmap='hot', vmin=0, vmax=1)
ax2.set_title("Column-Column Cosine Similarity", fontsize=14)
ax2.set_xlabel("Column Index")
ax2.set_ylabel("Column Index")
plt.colorbar(im2, ax=ax2, label="Similarity Score (0 to 1)")

plt.tight_layout()
plt.show()

Two graphs above suggests that the rows and columns of this picture are all pretty similar. For the sake of visualization, a stardandization procedure is needed

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


row_sim_no_diag = row_cos_sim.copy()
np.fill_diagonal(row_sim_no_diag, np.nan)
row_min_val = np.nanmin(row_sim_no_diag)


col_sim_no_diag = col_cos_sim.copy()
np.fill_diagonal(col_sim_no_diag, np.nan)
col_min_val = np.nanmin(col_sim_no_diag)

print(f"Row Similarity: [{row_min_val:.3f}, 1.0]")
print(f"Col Similarity: [{col_min_val:.3f}, 1.0]")



plt.figure(figsize=(16, 7),dpi=200)


ax1 = plt.subplot(1, 2, 1)
im1 = ax1.imshow(row_cos_sim, cmap='hot', origin='lower',
                   vmin=row_min_val, vmax=1.0)

ax1.set_title(f"Row Similarity Range: [{row_min_val:.2f}, 1.0]", fontsize=14)
ax1.set_xlabel("Row Index (0-447)")
ax1.set_ylabel("Row Index (0-447)")
plt.colorbar(im1, ax=ax1, label="Similarity Score")

ax2 = plt.subplot(1, 2, 2)
im2 = ax2.imshow(col_cos_sim, cmap='hot', origin='lower',
                   vmin=col_min_val, vmax=1.0)

ax2.set_title(f"Col Similarity Range: [{col_min_val:.2f}, 1.0]", fontsize=14)
ax2.set_xlabel("Column Index (0-671)")
ax2.set_ylabel("Column Index (0-671)")
plt.colorbar(im2, ax=ax2, label="Similarity Score")

plt.tight_layout()
plt.show()

###The Gramian Matrices: M@M.T and M.T@M
In linear algebra, a Gramian matrix (or Gram matrix) is a matrix where every entry is the inner product (dot product) of a set of vectors.In our project, img_float is our matrix M. The two similarity maps we calculated in cell [18] are precisely the two Gramian matrices of M.1. MMT (Row Similarity Map)Calculation: MMT = img_float @ img_float.TWhat it is: This is the Gramian matrix of the row vectors of M.Shape: (m, n) @ (n, m) = (m, m). In our case, (448, 448).Meaning: The value at MMT[i, j] is the dot product of row i and row j of the original image.Interpretation: This matrix shows the similarity between every pair of rows. A bright spot at (i, j) means row i and row j are highly similar (considering both their pattern and brightness).2. MTM (Column Similarity Map)Calculation: MTM = img_float.T @ img_floatWhat it is: This is the Gramian matrix of the column vectors of M.Shape: (n, m) @ (m, n) = (n, n). In our case, (672, 672).Meaning: The value at MTM[i, j] is the dot product of column i and column j of the original image.Interpretation: This matrix shows the similarity between every pair of columns.Connection to SVD (The Key Insight)These two matrices are not just for show; they are the mathematical foundation for finding the SVD matrices $U$ and $V$.The eigenvectors of MMT ($MM^T$) form the columns of the $U$ matrix (the left singular vectors).The eigenvectors of MTM ($M^TM$) form the columns of the $V$ matrix (the right singular vectors).The eigenvalues of both matrices are the squares of the singular values ($\sigma_k^2$).

### Use a 'toy' version of the original image to illustrate the process

In [ ]:
target_width = 18
target_height = 12
target_size = (target_width, target_height) # (width, height)
# The Pillow .resize() function takes (width, height) as input.

# Resize the grayscale_image directly
img_small = grayscale_image.resize(target_size, Image.Resampling.LANCZOS)

img_small_array = np.array(img_small)
array_small = img_small_array / 255.0

print(f"Created new matrix 'array_small'")
print(f"Shape of img_small_array: {array_small.shape}") # Should be (32, 48)

plt.figure(figsize=(8, 6)) # figsize=(width, height)
plt.imshow(array_small, cmap='gray')
plt.title(f"32x48 Image (array_small)")
plt.xlabel("Columns (n=48)")
plt.ylabel("Rows (m=32)")
plt.show()

# Set numpy to print cleanly for future steps
np.set_printoptions(precision=3, suppress=True)

In [ ]:
print("Displaying the full array_small matrix (12x18 pixels):\n")

# Temporarily change print options to display the full matrix without truncation
# 'threshold=np.inf' prevents numpy from truncating the output
# 'linewidth=150' ensures that lines don't wrap too early for wider matrices
# 'suppress=False' ensures that small numbers are not printed as 0
with np.printoptions(threshold=np.inf, linewidth=150, suppress=False):
    print(array_small)

# print("\nRestoring default numpy print options.")
# You can reset print options to default if needed
# np.set_printoptions(edgeitems=3, infstr='inf', linewidth=75, nanstr='nan', precision=8, suppress=False, threshold=1000, formatter=None)

In [ ]:
import sympy as sp
sp.init_printing() # Initialize sympy for pretty printing
A=sp.Matrix(array_small)
sp.pprint(A.evalf(2), wrap_line=False) # or A.n()

In [ ]:
M_M_T = array_small @ array_small.T # S_l, row similarity
# Shape: (16, 24) @ (24, 16) = (16, 16)

M_T_M = array_small.T @ array_small # S_r, column similarity
# Shape: (24, 16) @ (16, 24) = (24, 24)

print(f"Calculated M @ M.T (shape: {M_M_T.shape})")
print(f"Calculated M.T @ M (shape: {M_T_M.shape})")


print("\nDisplaying the full M_M_T matrix (Row Similarity):\n")
with np.printoptions(threshold=np.inf, linewidth=150, suppress=False):
    print(M_M_T)

print("\n" + "-" * 50 + "\n") # Separator for clarity

print("Displaying the full M_T_M matrix (Column Similarity):\n")
with np.printoptions(threshold=np.inf, linewidth=150, suppress=False):
    print(M_T_M)

# Note: numpy print options are temporarily set within the 'with' block
# and revert to their previous state afterwards.

In [ ]:
S_left=sp.Matrix(M_M_T)
S_right=sp.Matrix(M_T_M)
print("S_left matrix\n")
sp.pprint(S_left.evalf(2), wrap_line=False)
print("\nS_right matrix\n")
sp.pprint(S_right.evalf(2), wrap_line=False)

### Finding U and σ from M @ M.T  and Finding V from M.T @ M

In [ ]:
# Find the eigenvalues (λ) and eigenvectors of the 16x16 matrix
# np.linalg.eigh is used for symmetric matrices (like M @ M.T)
# The eigenvalues may not be sorted.
lambdas_U, U_manual = np.linalg.eigh(M_M_T)

# The eigenvectors are our U matrix.
# The eigenvalues (λ) are the *square* of the singular values (σ).
# Sort them in descending order (largest first)
idx_U = np.argsort(lambdas_U)[::-1] # Get sorting indices
lambdas_U = lambdas_U[idx_U]
U_manual = U_manual[:, idx_U]

# Get the singular values by taking the square root
s_manual_from_U = np.sqrt(lambdas_U)

print("Eigenvectors from (M @ M.T) give us the U matrix.")
print("The square root of the Eigenvalues (λ) give us the Singular Values (σ).")


# -----------------------------------------------------------------
print("-----------------------------------------------------------")
# Find the eigenvalues (λ) and eigenvectors of the 24x24 matrix
lambdas_V, V_manual = np.linalg.eigh(M_T_M)

# Sort them in descending order
idx_V = np.argsort(lambdas_V)[::-1]
lambdas_V = lambdas_V[idx_V]
V_manual = V_manual[:, idx_V]

# The eigenvectors are our V matrix.
Vh_manual = V_manual.T # Transpose to get Vh
print("Eigenvectors from (M.T @ M) give us the V matrix (Vh is its transpose).")

### Displaying the full U_manual, s_manual_from_U, and Vh-manual matrix:

In [ ]:
print("Displaying the full U_manual matrix:\n")
with np.printoptions(threshold=np.inf, linewidth=150, suppress=False):
    print(U_manual)

print("\n" + "-" * 50 + "\n")

print("Displaying the full s_manual_from_U (singular values) vector:\n")
with np.printoptions(threshold=np.inf, linewidth=150, suppress=False):
    print(s_manual_from_U)

print("\n" + "-" * 50 + "\n")

print("Displaying the full Vh_manual matrix:\n")
with np.printoptions(threshold=np.inf, linewidth=150, suppress=False):
    print(Vh_manual)


In [ ]:
# --- 1. Get the NumPy arrays ---
# (Assuming U_manual, s_manual_from_U, and Vh_manual exist from cell [38])
m, n = 12, 18

# --- 2. Create FULL SymPy Matrices (U, \u03a3, V^T) ---
# These correspond to the full mathematical SVD where U and V are square.

# U_full (m x m) = (12x12)
U_sp_full = sp.Matrix(U_manual)

# \u03a3_full (m x n) = (12x18)
s_diag_part = sp.diag(*[sp.sympify(s) for s in s_manual_from_U])
padding_zeros = sp.zeros(m, n - m)
Sigma_sp_full = s_diag_part.row_join(padding_zeros)

# V^T_full (n x n) = (18x18)
Vt_sp_full = sp.Matrix(Vh_manual)


# --- 3. Create REDUCED SymPy Matrices (U_r, \u03a3_r, V_r^T) ---
# These correspond to the "economy" SVD (what np.linalg.svd returns with full_matrices=False)

# U_reduced (m x m) = (12x12)
# For m < n, the reduced U is the same as the full U.
U_sp_reduced = sp.Matrix(U_manual)

# \u03a3_reduced (m x m) = (12x12)
# This is the square diagonal matrix of singular values.
Sigma_sp_reduced = s_diag_part # We already created this: sp.diag(...)

# V^T_reduced (m x n) = (12x18)
# We take the first 'm' (which is 12) rows from the full Vh_manual.
Vt_sp_reduced = sp.Matrix(Vh_manual[:m, :])


# --- 4. Print them in pairs for comparison ---

print("\nU (Full) (12x12 SymPy Matrix):\n")
sp.pprint(U_sp_full.evalf(2), wrap_line=False)
print("\nU (Reduced) (12x12 SymPy Matrix):\n")
print("\n(Note: For m < n, reduced U is the same as full U)\n")
sp.pprint(U_sp_reduced.evalf(2), wrap_line=False)

print("\n" + "="*70 + "\n")

print("\u03a3 (Full) (12x18 SymPy Matrix):\n")
sp.pprint(Sigma_sp_full.evalf(2), wrap_line=False)
print("\n\u03a3 (Reduced) (12x12 SymPy Matrix):\n")
sp.pprint(Sigma_sp_reduced.evalf(2), wrap_line=False)

print("\n" + "="*70 + "\n")

print("\nV^T (Full) (18x18 SymPy Matrix):\n")
sp.pprint(Vt_sp_full.evalf(2), wrap_line=False)
print("\nV^T (Reduced) (12x18 SymPy Matrix):\n")
print("\n(Note: This is the first 12 rows of the full V^T)\n")
sp.pprint(Vt_sp_reduced.evalf(2), wrap_line=False)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# --- 1. Visualize Row and Column Similarity ---
# (Uses M_M_T and M_T_M from cell [32])

print("--- Visualizing Similarity Matrices ---")
plt.figure(figsize=(16, 7))
plt.style.use('default') # Reset style for good heatmap colors

# --- Plot 1: Row Similarity (M @ M.T) ---
ax1 = plt.subplot(1, 2, 1)
# 'hot' colormap: bright means high similarity
im1 = ax1.imshow(M_M_T, cmap='hot', interpolation='nearest')
ax1.set_title(f"Row Similarity (M @ M.T)\nShape: {M_M_T.shape}", fontsize=14)
ax1.set_xlabel("Row Index (0-11)")
ax1.set_ylabel("Row Index (0-11)")
plt.colorbar(im1, ax=ax1, label="Similarity Score (Dot Product)")
# Note: The bright diagonal shows a row is perfectly similar to itself.
# Bright spots off-diagonal (e.g., at (0,1)) mean row 0 and row 1 are very similar.

# --- Plot 2: Column Similarity (M.T @ M) ---
ax2 = plt.subplot(1, 2, 2)
im2 = ax2.imshow(M_T_M, cmap='hot', interpolation='nearest')
ax2.set_title(f"Column Similarity (M.T @ M)\nShape: {M_T_M.shape}", fontsize=14)
ax2.set_xlabel("Column Index (0-17)")
ax2.set_ylabel("Column Index (0-17)")
plt.colorbar(im2, ax=ax2, label="Similarity Score (Dot Product)")
# Note: The bright blocks here show which columns are similar.
# For example, the right-most columns (16, 17) are very bright,
# matching the bright sky in the original image.

plt.tight_layout()
plt.show()


# --- 2. Visualize U (Eigen-rows) ---
# (Uses U_manual from cell [38])

num_patterns_to_show = 4 # Show the first 4

print("\n--- Visualizing U: The 'Eigen-row' Patterns (Columns of U) ---")
plt.figure(figsize=(10, 5))
plt.suptitle("U: Eigen-row Patterns (Columns of U)", fontsize=16, y=1.03)

for k in range(num_patterns_to_show):
    ax = plt.subplot(1, num_patterns_to_show, k + 1)

    # Get the k-th column (vector u_k) from U_manual
    u_k_vector = U_manual[:, k] # This is (12,)

    # Reshape to (12, 1) to plot as a vertical 12x1 "image"
    u_k_image = u_k_vector.reshape(12, 1)

    # We use 'seismic' colormap: Red is positive, Blue is negative, White is zero.
    # This shows which rows are "positive" and which are "negative" in this pattern.
    ax.imshow(u_k_image, cmap='seismic', aspect=0.15, vmin=-1, vmax=1)
    ax.set_title(f"u_{k+1} (k={k})")
    ax.set_yticks(np.arange(12)) # Label all 12 rows
    ax.set_yticklabels(np.arange(1, 13)) # Human-readable 1-12
    ax.set_xticks([]) # No x-axis needed

plt.tight_layout()
plt.show()
# Interpretation: u_1 (k=0) is the most dominant pattern. It's all one color,
# meaning all rows "activate" together to create the average brightness.
# u_2 (k=1) shows a top-bottom split (e.g., top rows positive, bottom rows negative),
# representing the main vertical gradient (sky vs. ground) in the image.


# --- 3. Visualize Vh (Eigen-columns) ---
# (Uses Vh_manual from cell [38])

print("\n--- Visualizing Vh: The 'Eigen-column' Patterns (Rows of Vh) ---")
plt.figure(figsize=(12, 7))
plt.suptitle("Vh: Eigen-column Patterns (Rows of Vh)", fontsize=16, y=1.03)

for k in range(num_patterns_to_show):
    ax = plt.subplot(num_patterns_to_show, 1, k + 1)

    # Get the k-th row (vector v_k_T) from Vh_manual
    v_k_T_vector = Vh_manual[k, :] # This is (18,)

    # Reshape to (1, 18) to plot as a horizontal 1x18 "image"
    v_k_T_image = v_k_T_vector.reshape(1, 18)

    ax.imshow(v_k_T_image, cmap='seismic', aspect=0.8, vmin=-1, vmax=1)
    ax.set_title(f"v_{k+1}^T (k={k})")
    ax.set_xticks(np.arange(18)) # Label all 18 columns
    ax.set_xticklabels(np.arange(1, 19)) # Human-readable 1-18
    ax.set_yticks([]) # No y-axis needed

plt.tight_layout()
plt.show()
# Interpretation: v_1^T (k=0) is the most dominant column pattern.
# It's all one color, meaning all columns activate together.
# v_2^T (k=1) shows a left-right split, representing the main
# horizontal gradient in the image.

### Step : creating rank-1 matrices

In [ ]:
print("\nVisualizing Individual Rank-1 Patterns")

def get_rank_1_pattern(k_index, U, s, Vh):
    """
    Calculates a single rank-1 pattern for a given k-index.

    Parameters:
        k_index (int): 0-based index (e.g., k=1 corresponds to index 0).
        U (np.array): U matrix obtained from SVD decomposition.
        s (np.array): 1D vector of singular values.
        Vh (np.array): Vh matrix (V^T) obtained from SVD decomposition.

    Returns:
        np.array: (m, n) rank-1 matrix of the corresponding pattern.
        float: Corresponding singular value (magnitude).
    """
    # Get the singular value (i.e., "weight" or "magnitude")
    sigma_k = s[k_index]

    # Numpy slicing: U[rows, columns]
    # Get the k-th left singular vector (and keep it as a 2D column vector)
    u_k = U[:, k_index:k_index+1] # start from k_index and end in k_index+1 but not included, maintinging the form of 2d column vector

    # Get the k-th right singular vector (and keep it as a 2D row vector)
    v_k_T = Vh[k_index:k_index+1, :]
    # Calculate the outer product and multiply by the weight
    pattern_k = sigma_k * (u_k @ v_k_T)

    return pattern_k, sigma_k



IMAGES_PER_FIGURE = 10
TOTAL_IMAGES = 50

print(f"\nVisualize the first {TOTAL_IMAGES} Rank-1 Patterns")
print(f"Opening {TOTAL_IMAGES // IMAGES_PER_FIGURE} figures, each containing {IMAGES_PER_FIGURE} images.")


for k_start_index in range(0, TOTAL_IMAGES, IMAGES_PER_FIGURE):

    fig, axes = plt.subplots(2, 5, figsize=(20, 8), dpi=300)
    axes = axes.flatten()
    k_end_rank = k_start_index + IMAGES_PER_FIGURE
    fig.suptitle(f"Deconstruction: Individual Patterns k={k_start_index + 1} to k={k_end_rank}",
                 fontsize=20, y=1)

    for i in range(IMAGES_PER_FIGURE):
        # Calculate the current k index
        k_index = k_start_index + i
        # Check if it's out of range for 50
        if k_index >= TOTAL_IMAGES:
            axes[i].axis('off') # Hide superfluous empty subplots
            continue


        pattern, sigma = get_rank_1_pattern(k_index, U_reduced, s_reduced, Vh_reduced)
        ax = axes[i]

        ax.imshow(pattern, cmap='gray')

        ax.set_title(f"Rank-1 Pattern for k={k_index + 1}\n(σ = {sigma:.2f})", fontsize=15)
        ax.axis('on')

    plt.tight_layout()
    plt.show()

print("All 50 patterns displayed.")

element-wise addition

In [ ]:
# Add a numerical representation plot for each rank 1 pattern here

图片一开始是[0,1]之间的，但是rank-1 matrices这些组件不一定每个像素的值都要是在[0,1]之间，可以为负数，可以大于1，但是最终相加后像素亮度值一定要在[0,1]之间

In [ ]:
def get_rank_1_pattern(k_index, U, s, Vh):
    """
    Calculates the individual rank-1 pattern for a given k-index.
    """
    sigma_k = s[k_index]
    u_k = U[:, k_index:k_index+1]
    v_k_T = Vh[k_index:k_index+1, :]
    pattern_k = sigma_k * (u_k @ v_k_T)
    return pattern_k, sigma_k


IMAGES_PER_FIGURE = 10
TOTAL_IMAGES_TO_SHOW = 50 # <---

m, n = img_float.shape
M_reconstructed = np.zeros((m, n)) # initialization

print(f"\nVisualizing Top {TOTAL_IMAGES_TO_SHOW} CUMULATIVE Reconstructions")
print(f"This will open {TOTAL_IMAGES_TO_SHOW // IMAGES_PER_FIGURE} figures, each with {IMAGES_PER_FIGURE} images.")
print("Each image is the SUM of all patterns before it.")

for k_start_index in range(0, TOTAL_IMAGES_TO_SHOW, IMAGES_PER_FIGURE):

    fig, axes = plt.subplots(2, 5, figsize=(20, 9),dpi=600)
    axes = axes.flatten() # Make the 2x5 grid easy to index

    k_end_rank = k_start_index + IMAGES_PER_FIGURE
    fig.suptitle(f"Cumulative Reconstruction: k={k_start_index + 1} to k={k_end_rank}",
                 fontsize=20, y=1.03)

    for i in range(IMAGES_PER_FIGURE):
        k_index = k_start_index + i
        # Get the next "chisel" (the next rank-1 pattern)
        pattern, sigma = get_rank_1_pattern(k_index, U_reduced, s_reduced, Vh_reduced)
        # "cumulative" part (pattern_1 + pattern_2 + ...)
        M_reconstructed = M_reconstructed + pattern


        ax = axes[i]
        ax.imshow(np.clip(M_reconstructed, 0, 1), cmap='gray')
        ax.set_title(f"Cumulative k = {k_index + 1}\n(Added σ = {sigma:.2f})", fontsize=16)
        ax.axis('off')

    plt.tight_layout()
    plt.show()

print(f"Finished showing all {TOTAL_IMAGES_TO_SHOW} cumulative reconstructions.")

In [ ]:
print("Pre-calculating all 448 cumulative reconstructions...")


# We need the U, s, Vh components
m, n = img_float.shape
U_reduced, s_reduced, Vh_reduced = np.linalg.svd(img_float, full_matrices=False)

precalculated_images = [] # <---store all the images
M_reconstructed = np.zeros((m, n))

for k_index in range(len(s_reduced)): # Loop from 0 to 447

    # Get the next rank-1 pattern
    sigma_k = s_reduced[k_index]
    u_k = U_reduced[:, k_index:k_index+1]
    v_k_T = Vh_reduced[k_index:k_index+1, :]

    pattern = sigma_k * (u_k @ v_k_T)

    M_reconstructed += pattern # each pattern is a rank-1 matrix
    precalculated_images.append(np.clip(M_reconstructed, 0, 1))

print(f"Done! Stored {len(precalculated_images)} images.")

In [ ]:
import plotly.graph_objects as go

m,n=img_float.shape
original_storage=m*n
total_energy=np.sum(s_reduced**2) # the square of the sum of singular values


fig=go.Figure()
for k,img in enumerate(precalculated_images):
  fig.add_trace(
      go.Heatmap(
          z=img,
          visible=(k==0),
          showscale=False,
          colorscale='gray',
      )
  )

steps=[]
# Creating instructions list for slider
num_images=len(precalculated_images) # 448
for k in range(num_images):
  num_components = k + 1
  k_storage=num_components*(m+n+1)
  compression_ratio=k_storage / original_storage
  compression_str = f"Storage Ratio: {compression_ratio:.2%}"

  lost_energy = np.sum(s_reduced[num_components:]**2)
  distortion_rate = lost_energy / total_energy
  distortion_str = f"Distortion: {distortion_rate:.2%}"

  step = dict(
        method="update",
        args=[
            {"visible": [False] * num_images},
            {"title": f"<b>Reconstruction: k = {num_components}</b><br>{compression_str} | {distortion_str}"}
        ],
        label=str(num_components)
    )
  step["args"][0]["visible"][k] = True
  steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "k = "},
    pad={"t": 50},
    steps=steps
)]

# Set default ratio and rate
default_comp_ratio = (1 * (m + n + 1)) / original_storage
default_dist_rate = np.sum(s_reduced[1:]**2) / total_energy

fig.update_layout(
    sliders=sliders,
    title=f"<b>Image Reconstruction: k = 1</b><br>Storage Ratio: {default_comp_ratio:.2%} | Distortion: {default_dist_rate:.2%}",
    width=800,
    height=700,
    yaxis=dict(autorange='reversed')
)

fig.show()
